In [4]:
import torch
import torch.nn.init

device = None
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# 랜덤 시드 고정
torch.cuda.manual_seed_all(777)

# 현재 디바이스가 무엇인지?
print('Current Device : ' + str(device))

Current Device : mps


In [16]:
import os
import torchvision.transforms as transforms # MNIST 데이터를 텐서 형태로 바꾸기 위해
import torchvision.datasets as dsets # 이 안에 MNIST 있다

if os.path.exists('MNIST'):
    print('There is MNIST data.')
    train_raw = dsets.MNIST(root=str(os.getcwd()), train=True, transform=transforms.ToTensor(), download=False)
    test_raw = dsets.MNIST(root=str(os.getcwd()), train=False, transform=transforms.ToTensor(), download=False)
    # train_raw = dsets.MNIST(root=str(os.getcwd()), train=True, download=False)
    # test_raw = dsets.MNIST(root=str(os.getcwd()), train=False, download=False)
else: # MNIST 데이터 다운로드
    print('There is no MNIST data.')
    train_raw = dsets.MNIST(root=str(os.getcwd()), train=True, transform=transforms.ToTensor(), download=True)
    test_raw = dsets.MNIST(root=str(os.getcwd()), train=False, transform=transforms.ToTensor(), download=True)

print(train_raw)
print(test_raw)

There is MNIST data.
Dataset MNIST
    Number of datapoints: 60000
    Root location: /Users/leejunseo/NODE
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: /Users/leejunseo/NODE
    Split: Test
    StandardTransform
Transform: ToTensor()


In [42]:
# t = train_raw.__getitem__(0)
# for idx, item in enumerate(train_raw):
#     print(item)
#     print(idx)
#     break
type((train_raw[0][0]/255)[0][0][0].item())

float

In [20]:
from torch.utils.data import Dataset

class Customdataset(Dataset):
    def __init__(self, x:dsets.mnist.MNIST, y:dsets.mnist.MNIST) -> None:
        self.x = x
        self.y = y
        return None

    def change_scale(self, tensor:torch.Tensor) -> torch.Tensor: # original MNIST 0~255 int -> 0~1 float로 변경
        return tensor/255 
    
    def quantize_tensor(self, tensor:torch.Tensor, levels:int) -> torch.Tensor:
        min_value = tensor.min()
        max_value = tensor.max()
        quantized_tensor = tensor.clone()
        quantized_tensor.sub_(min_value).div_(max_value - min_value + 1e-5).mul_(levels - 1).round_().div_(levels - 1).mul_(max_value - min_value + 1e-5).add_(min_value)
        return quantized_tensor

    def __len__(self):
        return len(self.x) # x 길이랑 y 길이랑 같음.

    def __getitem__(self, idx):
        return torch.from_numpy(self.x[idx]), torch.from_numpy(self.y[idx])

numpy.ndarray